# Burgers TensorFlow Implentation

Based on:

- https://github.com/maziarraissi/PINNs/tree/master/appendix/continuous_time_inference%20(Burgers)
- https://people.sc.fsu.edu/~jburkardt/f_src/burgers_solution/burgers_solution.html

In [13]:
%%writefile burgers-tf1.py
# TensorFlow 1.15
from time import time
t0 = time()
import tensorflow as tf
import logging, os
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
np.random.seed(1234)
tf.compat.v1.set_random_seed(1234)

DATA_BURGERS = 'burgers_shock.mat'

data = scipy.io.loadmat(DATA_BURGERS)
Exact = np.real(data['usol']).T

t = data['t'].flatten()[:, None]
x = data['x'].flatten()[:, None]


class PhysicsInformedNN:

    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        self.lb = lb
        self.ub = ub
        self.x_u = X_u[:, 0:1]
        self.t_u = X_u[:, 1:2]
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]
        self.u = u
        self.layers = layers
        self.nu = nu

        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)

        # tf placeholders and graph
        self.sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
            allow_soft_placement=True))
        self.x_u_tf = tf.compat.v1.placeholder(tf.float32,
                                               shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.compat.v1.placeholder(tf.float32,
                                               shape=[None, self.t_u.shape[1]])
        self.u_tf = tf.compat.v1.placeholder(tf.float32,
                                             shape=[None, self.u.shape[1]])
        self.x_f_tf = tf.compat.v1.placeholder(tf.float32,
                                               shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.compat.v1.placeholder(tf.float32,
                                               shape=[None, self.t_f.shape[1]])
        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf)
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(
            self.loss,
            method='L-BFGS-B',
            options={
                'maxiter': 50000,
                'maxfun': 50000,
                'maxcor': 50,
                'maxls': 50,
                'ftol': 1.0 * np.finfo(float).eps
            })
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l + 1]])
            b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                            dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim],
                                                      stddev=xavier_stddev),
                           dtype=tf.float32)

    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(0, num_layers - 2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x, t], 1), self.weights, self.biases)
        return u

    def net_f(self, x, t):
        u = self.net_u(x, t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = u_t + u * u_x - self.nu * u_xx
        return f

    # def callback(self, loss):
    #     print('Loss:', loss)

    def train(self):
        tf_dict = {
            self.x_u_tf: self.x_u,
            self.t_u_tf: self.t_u,
            self.u_tf: self.u,
            self.x_f_tf: self.x_f,
            self.t_f_tf: self.t_f
        }
        self.optimizer.minimize(
            self.sess,
            feed_dict=tf_dict,
            fetches=[self.loss],
            # loss_callback=self.callback
        )

    def predict(self, X_star):
        u_star = self.sess.run(self.u_pred, {
            self.x_u_tf: X_star[:, 0:1],
            self.t_u_tf: X_star[:, 1:2]
        })
        f_star = self.sess.run(self.f_pred, {
            self.x_f_tf: X_star[:, 0:1],
            self.t_f_tf: X_star[:, 1:2]
        })
        return u_star, f_star


nu = 0.01 / np.pi
#N_u = 100
#N_f = 10000
N_u = 100
N_f = 100
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat(DATA_BURGERS)

t = data['t'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = np.real(data['usol']).T  # Exact
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]  # Exact

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

xx1 = np.hstack((X[0:1, :].T, T[0:1, :].T))
xx2 = np.hstack((X[:, 0:1], T[:, 0:1]))
xx3 = np.hstack((X[:, -1:], T[:, -1:]))
X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub - lb) * lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]

# Exact
uu1 = Exact[0:1, :].T
uu2 = Exact[:, 0:1]
uu3 = Exact[:, -1:]
u_train = np.vstack([uu1, uu2, uu3])
u_train = u_train[idx, :]  # Exact

model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)

t1 = time()
model.train()
t2 = time()
print(f"Train: {t2-t1:.4f}")

for _ in range(1000):
    u_pred, f_pred = model.predict(X_star)
t3 = time()
print(f"Predict x1000: {t3-t2:.4f}")
print(f"Elapsed: {t3-t0:.4f}")

Overwriting burgers-tf1.py


Predict x1:

In [12]:
%%time
! numactl -C 1 time python burgers-tf1.py

Train: 28.2204
Predict: 0.3404
Elapsed: 35.9391
32.33user 1.76system 0:36.40elapsed 93%CPU (0avgtext+0avgdata 370028maxresident)k
0inputs+8outputs (0major+243476minor)pagefaults 0swaps
CPU times: user 429 ms, sys: 88.7 ms, total: 517 ms
Wall time: 36.5 s


In [14]:
%%time
! time python burgers-tf1.py

Train: 19.9331
Predict: 0.2690
Elapsed: 27.0448

real	0m27.507s
user	0m56.078s
sys	0m6.237s
CPU times: user 384 ms, sys: 65.1 ms, total: 449 ms
Wall time: 27.6 s


In [14]:
%%bash
BASE=/scratch${HOME#/prj}/421
cp burgers-tf1.py burgers_shock.mat $BASE

In [15]:
%%writefile burgftf.srm
#!/bin/bash
#SBATCH --job-name burgftf      # SLURM_JOB_NAME
#SBATCH --partition cpu_dev     # SLURM_JOB_PARTITION
#SBATCH --nodes=1               # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=1     # SLURM_NTASKS_PER_NODE
#SBATCH --cpus-per-task=1       # SLURM_CPUS_PER_TASK  $OMP_THREADS
#SBATCH --time=00:05:00         # Limit execution time

# VARIABLES OF INTEREST IN THE SLURM ENVIRONMENT
# <https://slurm.schedmd.com/sbatch.html>
# SLURM_PROCID
#     The MPI rank (or relative process ID) of the current process.
# SLURM_LOCALID
#     Node local task ID for the process within a job.
# SLURM_NODEID
#     ID of the nodes allocated. 

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of tasks:' $SLURM_NTASKS
echo '- # of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# load the Python environment
SCR=/scratch${PWD#/prj}
BASE=/scratch${HOME#/prj}/miniconda3
source $BASE/etc/profile.d/conda.sh
conda activate
conda activate tf1
cd $SCR

# run
echo -n '<1. starting> ' && date
echo "OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK"
echo '-- output -----------------------------'
               
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
srun -n $SLURM_NTASKS -c $SLURM_CPUS_PER_TASK time python burgers-tf1.py
               
echo '-- end --------------------------------'
echo -n '<2. quit>                    ' && date

Overwriting burgftf.srm


In [1]:
import time

def runtest(tasks):
    sub = !sbatch --cpus-per-task={tasks} burgftf.srm
    print(sub[0], end='.')
    job = sub[0].replace('Submitted batch job ','')
    c = [job]
    while job in c:
        time.sleep(10)
        print(end='.')
        c = !squeue --job {job} --noheader --format "%i"
    print('')
    out = !echo /scratch${PWD#/prj}/slurm-
    %cat {out[0] + job}.out

## Predict 1x

In [5]:
runtest(1)

Submitted batch job 10724580.................
- Job ID: 10724580
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1487
<1. starting> Dom Dez 11 18:59:10 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Train: 31.4064
Predict: 0.3318
Elapsed: 143.0704
33.40user 3.06system 2:26.46elapsed 24%CPU (0avgtext+0avgdata 380160maxresident)k
1737424inputs+408outputs (114major+237124minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:01:36 -03 2022


In [6]:
runtest(2)

Submitted batch job 10724593................
- Job ID: 10724593
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 2
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Dom Dez 11 19:16:37 -03 2022
OMP_NUM_THREADS=2
-- output -----------------------------
Train: 25.9244
Predict: 0.3277
Elapsed: 128.3989
38.12user 2.95system 2:11.45elapsed 31%CPU (0avgtext+0avgdata 371952maxresident)k
891968inputs+8outputs (2major+192034minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:18:49 -03 2022


In [7]:
runtest(4)

Submitted batch job 10724596...............
- Job ID: 10724596
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1008
<1. starting> Dom Dez 11 19:20:52 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Train: 22.6933
Predict: 0.3363
Elapsed: 120.6783
45.49user 4.65system 2:04.14elapsed 40%CPU (0avgtext+0avgdata 382760maxresident)k
873824inputs+8outputs (0major+189603minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:22:58 -03 2022


In [8]:
runtest(8)

Submitted batch job 10724602..............
- Job ID: 10724602
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 8
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1008
<1. starting> Dom Dez 11 19:27:08 -03 2022
OMP_NUM_THREADS=8
-- output -----------------------------
Train: 24.4757
Predict: 0.3090
Elapsed: 114.6924
50.39user 6.24system 1:57.78elapsed 48%CPU (0avgtext+0avgdata 391344maxresident)k
892016inputs+8outputs (2major+204785minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:29:06 -03 2022


In [9]:
runtest(16)

Submitted batch job 10724603.............
- Job ID: 10724603
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 16
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1008
<1. starting> Dom Dez 11 19:29:20 -03 2022
OMP_NUM_THREADS=16
-- output -----------------------------
Train: 23.3984
Predict: 0.2872
Elapsed: 100.8578
55.81user 7.70system 1:43.72elapsed 61%CPU (0avgtext+0avgdata 415196maxresident)k
892016inputs+8outputs (4major+243106minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:31:04 -03 2022


In [10]:
runtest(24)

Submitted batch job 10724605..............
- Job ID: 10724605
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 24
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1008
<1. starting> Dom Dez 11 19:31:26 -03 2022
OMP_NUM_THREADS=24
-- output -----------------------------
Train: 23.6385
Predict: 0.2959
Elapsed: 111.0270
55.55user 7.46system 1:54.02elapsed 55%CPU (0avgtext+0avgdata 410664maxresident)k
891960inputs+8outputs (4major+237242minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:33:20 -03 2022


## Predict 1000x

In [17]:
runtest(1)

Submitted batch job 10724612.........................
- Job ID: 10724612
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1230
<1. starting> Dom Dez 11 19:40:35 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Train: 30.9801
Predict x1000: 100.8024
Elapsed: 225.5623
121.03user 14.32system 3:49.39elapsed 59%CPU (0avgtext+0avgdata 382996maxresident)k
891872inputs+8outputs (0major+6429673minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:44:25 -03 2022


In [2]:
runtest(4)

Submitted batch job 10725225....................
- Job ID: 10725225
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Seg Dez 12 09:31:36 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Train: 23.0093
Predict x1000: 46.5797
Elapsed: 171.1143
162.93user 29.31system 2:54.33elapsed 110%CPU (0avgtext+0avgdata 401944maxresident)k
891968inputs+8outputs (2major+9391715minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Seg Dez 12 09:34:30 -03 2022


In [18]:
runtest(8)

Submitted batch job 10724617.................
- Job ID: 10724617
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 8
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1422
<1. starting> Dom Dez 11 19:45:55 -03 2022
OMP_NUM_THREADS=8
-- output -----------------------------
Train: 22.5165
Predict x1000: 31.9668
Elapsed: 140.9470
182.81user 36.51system 2:24.27elapsed 152%CPU (0avgtext+0avgdata 420080maxresident)k
892024inputs+8outputs (13major+9344281minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:48:20 -03 2022


In [19]:
runtest(16)

Submitted batch job 10724620................
- Job ID: 10724620
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 16
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1255
<1. starting> Dom Dez 11 19:48:36 -03 2022
OMP_NUM_THREADS=16
-- output -----------------------------
Train: 23.9791
Predict x1000: 28.4343
Elapsed: 136.4645
216.18user 73.54system 2:19.46elapsed 207%CPU (0avgtext+0avgdata 481896maxresident)k
905232inputs+8outputs (6major+9828710minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:50:56 -03 2022


In [20]:
runtest(24)

Submitted batch job 10724621................
- Job ID: 10724621
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 24
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1255
<1. starting> Dom Dez 11 19:51:07 -03 2022
OMP_NUM_THREADS=24
-- output -----------------------------
Train: 24.0846
Predict x1000: 27.6478
Elapsed: 131.5511
245.69user 111.84system 2:14.48elapsed 265%CPU (0avgtext+0avgdata 494516maxresident)k
891960inputs+8outputs (3major+9659334minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 19:53:22 -03 2022


# Execution 2

In [2]:
runtest(1)
runtest(4)
runtest(8)
runtest(16)
runtest(24)

Submitted batch job 10724697........................
- Job ID: 10724697
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1001
<1. starting> Dom Dez 11 22:05:49 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Train: 31.1117
Predict x1000: 98.6229
Elapsed: 223.8540
120.97user 12.25system 3:46.78elapsed 58%CPU (0avgtext+0avgdata 383052maxresident)k
892016inputs+8outputs (2major+5494642minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 22:09:36 -03 2022
Submitted batch job 10724702.................
- Job ID: 10724702
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1001
<1. starting> Dom Dez 11 22:09:49 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Train: 22.20

# Execution 3

In [3]:
runtest(1)
runtest(4)
runtest(8)
runtest(16)
runtest(24)

Submitted batch job 10724719.....................
- Job ID: 10724719
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1422
<1. starting> Dom Dez 11 22:19:29 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Train: 28.8894
Predict x1000: 102.4999
Elapsed: 197.5665
121.08user 15.61system 3:20.40elapsed 68%CPU (0avgtext+0avgdata 380432maxresident)k
905208inputs+8outputs (2major+7298870minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 22:22:50 -03 2022
Submitted batch job 10724723...............
- Job ID: 10724723
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1009
<1. starting> Dom Dez 11 22:22:57 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Train: 21.1996
P

## Version

In [7]:
import tensorflow as tf
print(tf.__version__)

1.15.2
